In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds


# 划分数据集
# 60%(15,000)用于训练 40%（10,000）用于验证(validation)
train_validation_split = tfds.Split.TRAIN.subsplit([6, 4])

(train_data, val_data), test_data = tfds.load(
    name="imdb_reviews",
    split=(train_validation_split, tfds.Split.TEST),
    as_supervised=True)

# 数据格式
train_examples_batch, train_label_batch = next(iter(train_data.batch(10)))
print("数据的样式为：")
print(train_examples_batch)
print(train_label_batch)

数据的样式为：
tf.Tensor(
[b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story penned by a journalist rather than a novel

 b"This is the most depressing film I have ever seen. I first saw it as a child and even thinking about it now really upsets me. I know it was set in a time when life was hard and I know these people were poor and the crops were vital. Yes, I get all that. What I find hard to take is I can't remember one single light moment in the entire film. Maybe it was true to life, I don't know. I'm quite sure the acting was top notch and the direction and quality of filming etc etc was wonderful and I know that every film can't have a happy ending but as a family film it is dire in my opinion.<br /><br />I wouldn't recommend it to anyone who wants to be entertained by a film. I can't stress enough how this film affected me as a child. I was talking about it recently and all the sad memories came flooding back. I think it would have all but the heartless reaching for the Prozac."], shape=(10,), dtype=string)
tf.Tensor([1 1 1 1 1 1 0 1 1 0], shape=(10,), dtype=int64)


In [2]:
# 文本嵌入过程
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=400, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

# Build the model

In [3]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


# loss function and optimizer

In [5]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

In [6]:
history = model.fit(train_data.shuffle(1000).batch(512), epochs=20, validation_data=val_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================]: 1.7177 - accuracy: 0.472 - 1s 470ms/step - loss: 1.6365 - accuracy: 0.491 - 1s 342ms/step - loss: 1.6023 - accuracy: 0.487 - 1s 281ms/step - loss: 1.5537 - accuracy: 0.492 - 1s 241ms/step - loss: 1.5249 - accuracy: 0.493 - 1s 214ms/step - loss: 1.5108 - accuracy: 0.488 - 1s 194ms/step - loss: 1.4853 - accuracy: 0.490 - 1s 179ms/step - loss: 1.4615 - accuracy: 0.488 - 2s 168ms/step - loss: 1.4396 - accuracy: 0.487 - 2s 159ms/step - loss: 1.4113 - accuracy: 0.490 - 2s 151ms/step - loss: 1.3845 - accuracy: 0.490 - 2s 145ms/step - loss: 1.3627 - accuracy: 0.489 - 2s 140ms/step - loss: 1.3403 - accuracy: 0.490 - 2s 135ms/step - loss: 1.3199 - accuracy: 0.488 - 2s 132ms/step - loss: 1.2936 - accuracy: 0.490 - 2s 128ms/step - loss: 1.2695 - accuracy: 0.489 - 2s 125ms/step - loss: 1.2492 - accuracy: 0.488 - 2s 122ms/step - loss: 1.2281 - accuracy: 0.489 - 2s 120ms/step - loss: 1.2093 - accuracy: 0.488 - 2s 118ms/step - loss: 1.1905 - accuracy:

30/30 [==============================] - ETA: 3s - loss: 0.4449 - accuracy: 0.79 - ETA: 2s - loss: 0.4467 - accuracy: 0.80 - ETA: 1s - loss: 0.4283 - accuracy: 0.81 - ETA: 1s - loss: 0.4325 - accuracy: 0.81 - ETA: 1s - loss: 0.4284 - accuracy: 0.81 - ETA: 1s - loss: 0.4264 - accuracy: 0.81 - ETA: 1s - loss: 0.4316 - accuracy: 0.81 - ETA: 1s - loss: 0.4315 - accuracy: 0.81 - ETA: 0s - loss: 0.4308 - accuracy: 0.81 - ETA: 0s - loss: 0.4290 - accuracy: 0.81 - ETA: 0s - loss: 0.4295 - accuracy: 0.81 - ETA: 0s - loss: 0.4297 - accuracy: 0.81 - ETA: 0s - loss: 0.4300 - accuracy: 0.80 - ETA: 0s - loss: 0.4297 - accuracy: 0.80 - ETA: 0s - loss: 0.4309 - accuracy: 0.80 - ETA: 0s - loss: 0.4301 - accuracy: 0.80 - ETA: 0s - loss: 0.4292 - accuracy: 0.81 - ETA: 0s - loss: 0.4282 - accuracy: 0.81 - ETA: 0s - loss: 0.4295 - accuracy: 0.81 - 3s 116ms/step - loss: 0.4270 - accuracy: 0.8161 - val_loss: 0.4362 - val_accuracy: 0.8039
Epoch 10/20
30/30 [==============================] - ETA: 3s - loss: 0.

30/30 [==============================] - ETA: 3s - loss: 0.2290 - accuracy: 0.91 - ETA: 2s - loss: 0.2457 - accuracy: 0.90 - ETA: 1s - loss: 0.2466 - accuracy: 0.90 - ETA: 1s - loss: 0.2376 - accuracy: 0.91 - ETA: 1s - loss: 0.2364 - accuracy: 0.91 - ETA: 1s - loss: 0.2378 - accuracy: 0.91 - ETA: 1s - loss: 0.2387 - accuracy: 0.91 - ETA: 1s - loss: 0.2370 - accuracy: 0.91 - ETA: 1s - loss: 0.2384 - accuracy: 0.91 - ETA: 0s - loss: 0.2353 - accuracy: 0.91 - ETA: 0s - loss: 0.2344 - accuracy: 0.91 - ETA: 0s - loss: 0.2332 - accuracy: 0.91 - ETA: 0s - loss: 0.2345 - accuracy: 0.91 - ETA: 0s - loss: 0.2349 - accuracy: 0.91 - ETA: 0s - loss: 0.2356 - accuracy: 0.91 - ETA: 0s - loss: 0.2359 - accuracy: 0.91 - ETA: 0s - loss: 0.2348 - accuracy: 0.91 - ETA: 0s - loss: 0.2353 - accuracy: 0.91 - ETA: 0s - loss: 0.2337 - accuracy: 0.91 - 4s 119ms/step - loss: 0.2329 - accuracy: 0.9132 - val_loss: 0.3150 - val_accuracy: 0.8679
Epoch 18/20
30/30 [==============================] - ETA: 3s - loss: 0.

# evaluate the model

In [7]:
? model.evaluate

In [10]:
results = model.evaluate(test_data.batch(512), verbose=0)
for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" %(name, value))

loss: 0.321
accuracy: 0.863
